# Evaluate APIs, define DAO membership
- Who is a DAO member? Followers in Snapshot? Token holders? DeepDAO (active?) membership?
- In the broadest sense, DAO membership can include followers, who are also important community participants in a DAO 

In [1]:
from util import snapshot_api
import pandas as pd

# Members, wealth, and voting
- Pick a small (<1k), medium (<10k), and large DAO (10k+)

In [2]:
query = """
query {
  spaces(
    first: 10000
    orderBy: "followersCount"
    orderDirection: asc
  ) {
    id
    name
    followersCount
  }
}
"""

res = snapshot_api(query)
df_spaces = (
    pd.DataFrame(res['spaces'])
    .sort_values('followersCount', ascending = False)
)
df_spaces

<Response [200]>


,id,name,followersCount
586,ens.eth,ENS,69227
910,gitcoindao.eth,Gitcoin,55738
777,uniswap,Uniswap,41619
431,olympusdao.eth,OlympusDAO,23167
598,banklessvault.eth,Bankless DAO,22231
...,...,...,...
574,hegic.eth,Hegic,0
376,resistanceio.eth,resdx.io,0
840,kaiinutoken.eth,Kai Inu Community,0
155,hoedao.eth,HoeDAO,0


In [3]:
dao_small = df_spaces.query('250 < followersCount < 750').iloc[0]
dao_medium = df_spaces.query('1000 < followersCount < 2500').iloc[0]
dao_large = df_spaces.query('10000 < followersCount < 12000').iloc[0]
print(f'''
SMALL: {dao_small.id} ({dao_small.followersCount:,d}) 
MEDIUM: {dao_medium.id} ({dao_medium.followersCount:,d}) 
LARGE: {dao_large.id} ({dao_large.followersCount:,d}) 
''')



SMALL: apeswap-finance.eth (743) 
MEDIUM: quickvote.eth (2,241) 
LARGE: xdaistake.eth (11,390) 



- Get all members (if possible try different APIs)

In [4]:
def get_followers(space):
    followers = []
    i = 0
    while True:
        skip = (i) * 1000
        first = (i+1) * 1000 
        query = """
        query {
          follows(
            first: """ + str(first) + """,
            skip: """ + str(skip) + """,
            where: {space: """ + f'"{space}"' + """}
          ){
            id
          }
        }
        """
        res = snapshot_api(query)
        followers += res['follows']
        if len(res['follows']) < 1000:
            break
        i += 1

    df_followers = pd.DataFrame(followers)
    return df_followers

In [5]:
df_followers_small = get_followers(dao_small.id)
df_followers_medium = get_followers(dao_medium.id)
df_followers_large = get_followers(dao_large.id)

<Response [200]>


In [107]:
df_followers_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743 entries, 0 to 742
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      743 non-null    object
dtypes: object(1)
memory usage: 5.9+ KB


In [108]:
df_followers_medium.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2241 entries, 0 to 2240
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2241 non-null   object
dtypes: object(1)
memory usage: 17.6+ KB


In [109]:
df_followers_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11390 entries, 0 to 11389
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11390 non-null  object
dtypes: object(1)
memory usage: 89.1+ KB


- Order them by wealth in ETH

In [6]:
df_followers_small.head()

,id
0,0xb52c1c5bf2b82f5562d4232691afd60f0041c1674332...
1,0x66f947e0eda068ee69a21347bb700444de9d052b4dcd...
2,0x2dba5c73253a0d75c0a404fc47083c76b3a19725d3f1...
3,0x69bb87c2a5c5f2e1beeff5aa163827187e1e64b8cd46...
4,0x2810ef94fd4eedcd1d1b049adefbe447d67232bce592...


- Compute correlation ETH wealth and number of votes in the DAO